In [1]:
import pandas as pd 
import ast

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import tqdm

In [19]:

data_traiter = pd.read_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/Medicament_pharma.csv").dropna()

In [20]:
data_traiter['Code_EAN_13'] = data_traiter['Code_EAN_13'].astype(int)
data_traiter['Code_EAN_13'] = data_traiter['Code_EAN_13'].astype(str).str.replace(' ', '')

In [57]:

import pymysql

host = "srv1905.hstgr.io"
user = "u791384722_pharma"
password = "@sCM^n4L"
database = "u791384722_pharma"

       

In [55]:

request = """INSERT INTO Medicament (
                    Code_EAN_13,
                    Nom,
                    Image_URL,
                    Présentation,
                    Dosage,
                    Distributeur_ou_fabriquant,
                    Composition,
                    Classe_thérapeutique,
                    Statut,
                    Code_ATC,
                    PPV,
                    Prix_hospitalier,
                    Tableau,
                    Indications,
                    Min_Stock,
                    Stock_Actuel, 
                    url_medicament
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                );"""


In [7]:
import os
import json

def parcourir_dossier_json(chemin_dossier):
    list_of_data = []
    for fichier in os.listdir(chemin_dossier):
        if fichier.endswith(".json"):  # on ne prend que les JSON
            chemin_fichier = os.path.join(chemin_dossier, fichier)
            with open(chemin_fichier, "r", encoding="utf-8") as f:
                try:
                    values = json.load(f) 
                    Code_EAN_13 = values.get("Code_EAN_13")
                    Nom = values.get("Nom")
                    Image_URL = values.get("Image URL")
                    Présentation = values.get("Présentation")
                    Dosage = values.get("Dosage")
                    Distributeur_ou_fabriquant = values.get("Distributeur ou fabriquant")
                    Composition = values.get("Composition")
                    Classe_thérapeutique = values.get("Classe thérapeutique")
                    Statut = values.get("Statut")
                    Code_ATC = values.get("Code ATC")
                    PPV = values.get("PPV")
                    Prix_hospitalier = values.get("Prix hospitalier")
                    Tableau = values.get("Tableau")
                    Indications = values.get("Indication(s)")
                    Min_Stock = 0
                    Stock_Actuel = 0
                    url_medicament = values.get("url")
                    list_of_data.append(values)
                except json.JSONDecodeError:
                    print(f"Erreur : {fichier} n'est pas un JSON valide.")
    return pd.DataFrame(list_of_data)

# Exemple d'utilisation
data_scraped = parcourir_dossier_json("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/medicaments")

In [8]:
data_scraped = data_scraped[["Code_EAN_13", "Nom", "Image URL", "Présentation", "Dosage", "Distributeur ou fabriquant","Composition", "Classe thérapeutique",  "Statut","Code ATC"
                             , "PPV", "Prix hospitalier", "Tableau", "Indication(s)", "url"]]

In [31]:
list_new = []
for i in data_scraped['Code_EAN_13'].values:
    if i not in data_traiter['Code_EAN_13'].values:
        list_new.append(i)
        

In [46]:
data_to_add = data_scraped[data_scraped['Code_EAN_13'].isin(list_new)]

In [48]:
data_to_add.dropna(subset=['PPV'], inplace=True)

/var/folders/c0/8qnp310s191511w2thxjfqkr0000gn/T/ipykernel_81192/1271771800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_add.dropna(subset=['PPV'], inplace=True)


In [49]:
data_to_add.info()

<class 'pandas.core.frame.DataFrame'>
Index: 646 entries, 0 to 2657
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Code_EAN_13                 646 non-null    object 
 1   Nom                         646 non-null    object 
 2   Image URL                   646 non-null    object 
 3   Présentation                646 non-null    object 
 4   Dosage                      577 non-null    object 
 5   Distributeur ou fabriquant  646 non-null    object 
 6   Composition                 646 non-null    object 
 7   Classe thérapeutique        593 non-null    object 
 8   Statut                      646 non-null    object 
 9   Code ATC                    616 non-null    object 
 10  PPV                         646 non-null    float64
 11  Prix hospitalier            640 non-null    float64
 12  Tableau                     618 non-null    object 
 13  Indication(s)               541 non-nul

In [58]:
conn = pymysql.connect(
                    host=host, user=user, password=password, database=database
                )   
for index, values in data_to_add.iterrows():
    Code_EAN_13 = values.get("Code_EAN_13")
    Nom = values.get("Nom")
    Image_URL = values.get("Image URL")
    Présentation = values.get("Présentation")
    Dosage = values.get("Dosage") if pd.notna(values.get("Dosage")) else ""
    Distributeur_ou_fabriquant = values.get("Distributeur ou fabriquant")
    Composition = values.get("Composition")
    Classe_thérapeutique = values.get("Classe thérapeutique") if pd.notna(values.get("Classe thérapeutique")) else ""
    Statut = values.get("Statut")
    Code_ATC = values.get("Code ATC") if pd.notna(values.get("Code ATC")) else ""
    PPV = values.get("PPV")
    Prix_hospitalier = values.get("Prix hospitalier") if pd.notna(values.get("Prix hospitalier")) else 0
    Tableau = values.get("Tableau") if pd.notna(values.get("Tableau")) else ""
    Indications = values.get("Indication(s)") if pd.notna(values.get("Indication(s)")) else ""
    Min_Stock = 0
    Stock_Actuel = 0
    url_medicament = values.get("url")
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    cursor.execute(
        request,
        (
            Code_EAN_13,
            Nom,
            Image_URL,
            Présentation,
            Dosage,
            Distributeur_ou_fabriquant,
            Composition,
            Classe_thérapeutique,
            Statut,
            Code_ATC,
            PPV,
            Prix_hospitalier,
            Tableau,
            Indications,
            Min_Stock,
            Stock_Actuel,
            url_medicament,
        ),
    )
    conn.commit()

    

ProgrammingError: nan can not be used with MySQL